In [ ]:
from bids import BIDSLayout
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from pathlib import Path
import nibabel as nib
import numpy as np
from src.metrics import corr
from nilearn.glm import fdr_threshold
from scipy.stats import ttest_1samp, norm
import pandas as pd
import plotly.express as px

# Build mean EN

In [ ]:
lang = "EN"
n_runs = 9

In [ ]:
def average(run):
    layout = BIDSLayout("data/li/derivatives", validate=False, is_derivative=True)
    subjects = [s for s in layout.get_subjects() if s.startswith(lang)]
    a = None
    affine = None
    for subject in tqdm(subjects, desc=f"Run {run}", leave=False):
        imgs = sorted([f.path for f in layout.get(subject=subject)])
        assert len(imgs) == n_runs
        img = nib.load(imgs[run])
        if a is None:
            a = img.get_fdata()
            affine = img.affine
        else:
            a += img.get_fdata()
        img = layout.get(subject=subject)[0]
        new_path = img.path.replace("sub-"+subject, "mean"+lang)
        new_path = new_path.replace(f"run-{img.entities["run"]}", f"run-{run+1:02d}")
    a /= len(subjects)
    Path(new_path).parent.mkdir(parents=True, exist_ok=True)
    nib.Nifti1Image(a, affine).to_filename(new_path)
    print(new_path)

In [ ]:
for run in tqdm(range(n_runs)):
    average(run)

# ISC

In [ ]:
lang = "EN"

In [ ]:
layout = BIDSLayout("data/li2022/derivatives", validate=False, is_derivative=True)
subjects = layout.get_subjects()
subjects = [s for s in subjects if s.startswith(lang)]
n_subjects = len(subjects)

In [ ]:
def read(path):
    return path, nib.load(path).slicer[40:50, 40:55, 35:65].get_fdata()

In [ ]:
mean_imgs = sorted([f.path for f in layout.get(subject="meanEN")])
mean_imgs = sorted(process_map(read, mean_imgs[:3]))
mean_imgs = np.concatenate([m[1] for m in mean_imgs], axis=-1) * n_subjects

In [ ]:
corrs = []
for subject in tqdm(subjects):
    subject_imgs = sorted([f.path for f in layout.get(subject=subject)])
    subject_imgs = sorted(process_map(read, subject_imgs[:3], leave=False))
    subject_imgs = np.concatenate([m[1] for m in subject_imgs], axis=-1)
    c = corr(mean_imgs - subject_imgs, subject_imgs, axis=-1)
    corrs.append(c)
corrs = np.stack(corrs)

In [ ]:
pvalues = np.nan_to_num(ttest_1samp(np.arctanh(corrs), popmean=0, axis=0).pvalue, nan=1)
zscores = norm.ppf(1 - pvalues)
thresh = fdr_threshold(zscores.reshape(-1), 5e-2)
signif = np.where(zscores > thresh)

In [ ]:
df = pd.DataFrame(signif, index=["x", "y", "z"]).T
df["zscore"] = zscores[signif]

In [ ]:
import os
os.makedirs("figs", exist_ok=True)
px.scatter_3d(df, x="x", y="y", z="z", color="zscore", color_continuous_midpoint=0, opacity=.85).write_html("figs/li2022_ics.html")

In [ ]:
from nilearn import plotting

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
cmap = mpl.cm.jet
max_zscore = df.zscore.abs().max()
norm = mpl.colors.Normalize(vmin=-max_zscore, vmax=max_zscore)
cbar = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
colors = cmap(norm(df.zscore))

In [ ]:
colors

In [ ]:
plotting.view_connectome(adjacency_matrix=np.diag(df.zscore), node_coords=df[["x", "y", "z"]], node_color=colors, edge_cmap=cmap) #node_color=cmap(norm(df.zscore)))
#.save_as_html("figs/li2022_ics.html")